In [2]:
cd ..

/Users/vijay/vijaygkd/deep-ttt


In [438]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as k
import mlflow

In [6]:
#%load_ext autoreload
%autoreload 2

In [1089]:
from dqn.environment import *
from dqn.agent import RandomAgent, QAgent
from dqn.model import DQN
from dqn.train import Memory, QLearning, play_games_against_random


In [1224]:
t = QLearning()

hey11


In [1225]:
t.net.model.summary()

Model: "model_63"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_states (InputLayer)      [(None, 9)]          0           []                               
                                                                                                  
 hidden_1 (Dense)               (None, 128)          1280        ['input_states[0][0]']           
                                                                                                  
 hidden_2 (Dense)               (None, 64)           8256        ['hidden_1[0][0]']               
                                                                                                  
 input_actions (InputLayer)     [(None, 1)]          0           []                               
                                                                                           

In [1226]:
with mlflow.start_run(run_name='test run 11 - q value output layer', description='') as run:
    t.train(10000)

100%|█████████████████████████████████████████████████████████████████| 10000/10000 [1:01:44<00:00,  2.70it/s]


In [1316]:
X = np.array([[1., 0., 2.],
             [0., 1., 0.],
             [0., 0., 3.]])
Xt = tf.convert_to_tensor(X)

W = np.array([[1., 0.],
             [1., 1.],
             [0., 1.]])
Wt = tf.convert_to_tensor(W)

In [1324]:
Y_hat = np.dot(X, W)
Y_hat

array([[1., 2.],
       [1., 1.],
       [0., 3.]])

In [1325]:
Y = np.array([[2., 2.],
             [1., 3.],
             [0., 1.]])

In [1326]:
mse = tf.keras.losses.MeanSquaredError()
mse(Y, Y_hat).numpy()

1.5

In [1327]:
np.dot(X, Y-Y_hat) * -2

array([[-2.,  8.],
       [-0., -4.],
       [-0., 12.]])

In [1282]:
with tf.GradientTape(persistent=True) as g:
    g.watch([Xt, Wt])
    Yh = Xt @ Wt
    L = mse(Y, Yh)

In [1286]:
g.gradient(L, [Wt, Xt])

[<tf.Tensor: shape=(3, 2), dtype=float64, numpy=
 array([[-0.33333333,  0.        ],
        [ 0.33333333, -0.33333333],
        [-1.66666667,  2.        ]])>,
 <tf.Tensor: shape=(3, 3), dtype=float64, numpy=
 array([[-0.33333333, -0.33333333,  0.        ],
        [ 0.33333333,  0.        , -0.33333333],
        [-0.33333333,  0.33333333,  0.66666667]])>]

In [1285]:
L

<tf.Tensor: shape=(), dtype=float64, numpy=1.3333333333333333>

In [1299]:
np.dot(X.T, (Y-Y_hat)) * -2 / 6

array([[-0.33333333, -0.        ],
       [ 0.33333333, -0.33333333],
       [-1.66666667,  2.        ]])

In [1291]:
np.dot(Y_hat-Y, W.T) * 2 / 6

array([[-0.33333333, -0.33333333,  0.        ],
       [ 0.33333333,  0.        , -0.33333333],
       [-0.33333333,  0.33333333,  0.66666667]])

In [1303]:
np.dot(X, (Y-Y_hat)) * -2 / 6  # ---> Wrong

array([[-1.        ,  1.33333333],
       [ 0.33333333, -0.33333333],
       [-1.        ,  2.        ]])

In [737]:
y_true = [[0., 3.], [0., 0.], [0., 1.]]
y_pred = [[1., 0.], [1., 0.], [0., 1.]]
# Using 'auto'/'sum_over_batch_size' reduction type.
mse = tf.keras.losses.MeanSquaredError()
mse(y_true, y_pred).numpy()

1.8333334

In [729]:
y_true = [np.array([1, 6, 0, 2, 2]),
            np.array([ 1.208865  , -0.27224046,  0.09784675,  1.5510192 ,  2.])]

y_pred = [np.array([1, 6, 0, 2, 1]),
            np.array([ 1.208865  , -0.27224046,  0.09784675,  1.5510192 ,  0.])]

In [738]:
mse(ans, ans1).numpy()

8.297752

In [736]:
mse(ans, ans1).numpy()

array([10.84375  ,  5.7517548], dtype=float32)

In [701]:
y_pred

[array([1, 6, 0, 2, 2]),
 array([ 1.208865  , -0.27224046,  0.09784675,  1.5510192 ,  0.68674856])]

In [1173]:
t.train(50)

100%|█████████████████████████████████████████████████████████████████████████| 50/50 [00:03<00:00, 14.80it/s]


In [1174]:
board_samples = t.memory.sample_records(5)
board_samples

[array([[ 1.,  0.,  0.,  1.,  0.,  0., -1.,  0.,  0.],
        [ 0.,  0., -1.,  0.,  1.,  0.,  1.,  0., -1.],
        [ 0., -1.,  0.,  0.,  1.,  0.,  1.,  1., -1.],
        [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  1., -1.,  0.,  1.,  0.,  0., -1.,  0.]]),
 array([4., 7., 0., 2., 6.]),
 array([-0.1, -1. , -1. , -0.1, -1. ]),
 array([[ 1.,  1.,  0.,  1., -1.,  0., -1.,  0.,  0.],
        [-1.,  0., -1.,  0.,  1.,  0.,  1.,  1., -1.],
        [-1., -1.,  0.,  1.,  1.,  0.,  1.,  1., -1.],
        [ 0.,  0., -1.,  1.,  0.,  0.,  1.,  0.,  0.],
        [ 1.,  1., -1.,  1.,  1.,  0., -1., -1.,  0.]]),
 array([0., 0., 0., 0., 0.])]

In [ ]:
a

In [740]:
ans1 = t.net.model.predict([board_samples[0], board_samples[1]])
ans1

1/1 [==============================] - 0s 19ms/step


[array([3, 4, 1, 4, 5]),
 array([-0.14812343,  0.66704184,  0.0287344 , -0.09794629, -0.02111896],
       dtype=float32)]

In [1406]:
g = TicTacToe()

In [1412]:
g.execute_action(6)


(-1, 0)

In [1413]:
m, q = t.agent.play(g.get_current_state())
print(m, q)
g.execute_action(m)
g.print_board()

1 0.25119048
X X X 

- O X 

O - O 



In [1348]:
g.print_board()

- - - 

- - - 

- - - 



In [1161]:
s = play_games_against_random(t.agent, 100)

In [1162]:
s

{'win': 0.91, 'lost': 0.06, 'draw': 0.03}

In [1169]:
board_samples

[array([[ 1., -1.,  1., -1.,  0., -1.,  1.,  1., -1.]]),
 array([4.]),
 array([0.5]),
 array([[-1., -1., -1., -1., -1., -1., -1., -1., -1.]]),
 array([1.])]

### Test gradient updates